In [1]:
from load import Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
d = Dataset("cellcycle", nan_strategy="none")

In [3]:
def pad_hierarchical_labels_multi_label(labels):
    max_depth = max(max(len(path) for path in label) for label in labels)
    padded_labels = [[path + [""] * (max_depth - len(path)) for path in label] for label in labels]
    return np.array(padded_labels)

In [4]:
from sklearn.pipeline import Pipeline
from hiclass import MultiLabelLocalClassifierPerNode
from hiclass.metrics import f1
from sklearn.tree import DecisionTreeClassifier
from feature_selection import ModSelectKBest
from handle_nan import NumericImputer

pipeline = Pipeline([
    ("imputer", NumericImputer(strategy="mean")),
    ("selector", ModSelectKBest(k=10)),
    ("model", MultiLabelLocalClassifierPerNode(DecisionTreeClassifier()))
], verbose=True, memory="cache")

In [5]:
pipeline.fit(d.x_train(), d.y_train())
y_pred = pipeline.predict(d.x_test())

[Pipeline] ........... (step 1 of 3) Processing imputer, total=   0.0s


In [ ]:
y_true = d.y_test()

0                            [[14, 04], [20, 09, 07, 03]]
1                            [[10, 01, 09, 05], [42, 07]]
2                        [[01, 06, 10], [11, 02, 03, 04]]
3       [[01, 04], [10, 03, 01, 03], [10, 03, 03], [11...
4                                          [[42, 10, 03]]
                              ...                        
1276                 [[11, 02, 03, 01], [11, 02, 03, 04]]
1277    [[14, 07, 11, 01], [14, 13, 01, 01], [18, 02, ...
1278                                   [[11, 04, 03, 01]]
1279    [[14, 04], [14, 07, 11, 01], [14, 13, 01], [16...
1280       [[10, 01, 09, 05], [14, 07, 04], [42, 10, 03]]
Name: class, Length: 1281, dtype: object

In [ ]:
from feature_selection import fill_reshape

y_true_pad = fill_reshape(y_true)
y_pred_pad = fill_reshape(y_pred)

c:\Users\sjtom\Documents\MUNI\Semester jar 2024\PV056 - Machine Learning and Data Mining\hiclass\feature_selection.py:75: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  max_len = y.apply(len).agg(max)
c:\Users\sjtom\Documents\MUNI\Semester jar 2024\PV056 - Machine Learning and Data Mining\hiclass\feature_selection.py:76: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  depth = y.apply(lambda x: max([len(label) for label in x])).agg(max)


AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [ ]:
f1(y_true_pad, y_pred_pad)